# Clustering Crypto

In [51]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [52]:
# Load the crypto_data.csv dataset.
import pandas as pd

file_path = "../Datasets/crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [53]:
# Keep all the cryptocurrencies that are being traded.
crypto_df_tradeTrue = crypto_df[crypto_df["IsTrading"] == True]
crypto_df_tradeTrue["IsTrading"].unique()

array([ True])

In [55]:
# Drop the IsTrading column 
crypto_df_new = crypto_df_tradeTrue.drop(["IsTrading"], axis = 1)
crypto_df_new.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [5]:
# Remove rows that have at least 1 null value.
crypto_df_dropnas = crypto_df_new.dropna()
# len(crypto_df_dropnas)
# len(crypto_df_new)

In [47]:
# Keep the rows where coins are mined.
crypto_coins = crypto_df_dropnas[crypto_df_dropnas["TotalCoinsMined"] > 0.000000e+00]
crypto_coins.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [7]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df_clean = crypto_coins.set_index("Unnamed: 0")
crypto_df_clean.head(5)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [8]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names = crypto_df_clean["CoinName"]
crypto_names = pd.DataFrame(crypto_names)
crypto_names

,CoinName
Unnamed: 0,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex


In [9]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df_fin = crypto_df_clean.drop(["CoinName"], axis = 1)
crypto_df_fin.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000


In [10]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df_fin, columns = ["Algorithm", "ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(X)
crypto_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [12]:
# Using PCA to reduce dimension to three principal components.

# Initialize PCA model
pca = PCA(n_components=3)

# Get two principal components for the iris data.
X_pca = pca.fit_transform(X)

# The resulting principal components are transformed into a DataFrame to fit K-means:
#Transform PCA data to a DataFrame 
df_X_pca = pd.DataFrame(data=X_pca, columns=["principal component 1", "principal component 2", "principal component 3"])
df_X_pca.head()

,principal component 1,principal component 2,principal component 3
0,-1.162934e+10,-5.502574e+08,0.814728
1,-1.064088e+10,-1.197852e+09,0.814484
2,2.750507e+11,1.312370e+11,0.989919
3,-1.160219e+10,-5.552767e+08,-0.712858
4,-1.157548e+10,-6.435026e+08,-0.706031


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [13]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_X_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\adeve\anaconda3\envs\mlev\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  f"KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [36]:
# Initialize the K-means model
model = KMeans(n_clusters=3, random_state=1)

# Fit the model
model.fit(df_X_pca)

# Predict clusters
predictions = model.predict(df_X_pca)

# Add the predicted class columns
df_X_pca["class"] = model.labels_
df_X_pca.head()

,principal component 1,principal component 2,principal component 3,class
0,-1.162934e+10,-5.502574e+08,0.814728,0
1,-1.064088e+10,-1.197852e+09,0.814484,0
2,2.750507e+11,1.312370e+11,0.989919,0
3,-1.160219e+10,-5.552767e+08,-0.712858,0
4,-1.157548e+10,-6.435026e+08,-0.706031,0


In [50]:
df_X_pca["Unnamed:0"] = crypto_coins["Unnamed: 0"]
df_X_pca["Unnamed:0"].unique()

array(['42', nan, '404', '1337', 'BTC', 'ETH', 'LTC', 'DASH', 'XMR',
       'ETC', 'ZEC', 'BTS', 'DGB', 'BTCD', 'XPY', 'PRC', 'KOBO', 'SPR',
       'ARG', 'AUR', 'BLU', 'XMY', 'MOON', 'ZET', 'SXC', 'QTL', 'ENRG',
       'QRK', 'RIC', 'DGC', 'BTB', 'CAT', 'CBX', 'CCN', 'CRYPT', 'CSC',
       'DMD', 'XVG', 'DVC', 'EAC', 'EFL', 'EMC2', 'EMD', 'EXCL', 'FLT',
       'FRK', 'FTC', 'GDC', 'GLC', 'GLD', 'HBN', 'HYP', 'IFC', 'IOC',
       'IXC', 'KGC', 'LKY', 'LTB', 'MAX', 'MEC', 'MED', 'MINT', 'MINC',
       'MZC', 'NAUT', 'NAV', 'NOBL', 'NMC', 'NYAN', 'OPAL', 'ORB', 'POT',
       'PXC', 'RDD', 'RPC', 'SBC', 'SMC', 'SUPER', 'SYNC', 'SYS', 'TES',
       'TGC', 'TIT', 'TOR', 'TRC', 'UNB', 'UNO', 'URO', 'USDE', 'UTC',
       'VIA', 'VRC', 'VTC', 'WDC', 'XC', 'XCR', 'XJO', 'XST', 'ZCC',
       'BCN', 'XDN', 'BURST', 'SJCX', 'MONA', 'NTRN', 'FAIR', 'NLG',
       'RBY', 'PTC', 'KORE', 'WBB', 'NOTE', 'FLO', '8BIT', 'STV', 'ABY',
       'FLDC', 'U', 'UIS', 'CYP', 'OMC', 'VTR', 'GRE', 'XCN', 'MSC',
   

In [37]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df_fin, df_X_pca, crypto_names], axis = 1) 

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(1064, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,class,CoinName
0,NaN,NaN,NaN,NaN,-1.162934e+10,-5.502574e+08,0.814728,0.0,NaN
1,NaN,NaN,NaN,NaN,-1.064088e+10,-1.197852e+09,0.814484,0.0,NaN
2,NaN,NaN,NaN,NaN,2.750507e+11,1.312370e+11,0.989919,0.0,NaN
3,NaN,NaN,NaN,NaN,-1.160219e+10,-5.552767e+08,-0.712858,0.0,NaN
4,NaN,NaN,NaN,NaN,-1.157548e+10,-6.435026e+08,-0.706031,0.0,NaN
5,NaN,NaN,NaN,NaN,-1.152507e+10,-5.628276e+08,-0.574821,0.0,NaN
6,NaN,NaN,NaN,NaN,-1.160577e+10,-5.470735e+08,0.773963,0.0,NaN
7,NaN,NaN,NaN,NaN,-1.162074e+10,-5.651521e+08,-0.696025,0.0,NaN
8,NaN,NaN,NaN,NaN,-1.139080e+10,-5.433762e+08,-0.705865,0.0,NaN
9,NaN,NaN,NaN,NaN,-1.160746e+10,-5.461464e+08,-0.713007,0.0,NaN


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [16]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE


In [17]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

In [18]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [19]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [20]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

NameError: name 'plot_df' is not defined

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
